In [15]:
import re
import os
from collections import defaultdict

from PyPDF2 import PdfWriter, PdfReader
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

In [2]:
REPORTS_DIRECTORY = 'reports'
PAGES_DIRECTORY = 'pages'
PAGES_TXT_DIRECTORY = 'pages_txt'

RE_DERIVED = re.compile('\w+( -|- |-|! - )\w+')
RE_RUSSIAN_TEXT = re.compile("[а-яА-Я]+")
EXCLUDE_PATTERNS = re.compile("[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+")

stopwords_ru = stopwords.words("russian")

In [32]:
def reports_to_pages():
    errors = []
    for report in tqdm(os.listdir(REPORTS_DIRECTORY)):
        if not report.endswith('.pdf'):
            continue
        try:
            input_pdf = PdfReader(open(f'{REPORTS_DIRECTORY}/{report}', "rb"))
        except Exception:
            errors.append(report)
            continue
        _, report = report.rsplit('_', maxsplit=1)
        report = report.rstrip('.pdf')
        for index, page in enumerate(input_pdf.pages):
            output = PdfWriter()
            output.add_page(page)
            with open(f'{PAGES_DIRECTORY}/{report}_{index}.pdf', "wb") as outputStream:
                output.write(outputStream)
    print(*errors, sep=' ')

In [21]:
reports_to_pages()

 10%|███▊                                    | 134/1390 [17:27<42:00,  2.01s/it]Invalid stream (index 7) within object 2757 0: Stream has ended unexpectedly
Invalid stream (index 3) within object 789 0: Stream has ended unexpectedly
 11%|████▌                                   | 158/1390 [18:43<27:10,  1.32s/it]Multiple definitions in dictionary at byte 0x1069e2 for key /Im194
Multiple definitions in dictionary at byte 0x106a00 for key /Im194
Multiple definitions in dictionary at byte 0x106a1e for key /Im194
Multiple definitions in dictionary at byte 0x106a3c for key /Im194
Multiple definitions in dictionary at byte 0x106a5a for key /Im194
Multiple definitions in dictionary at byte 0x106a78 for key /Im194
Multiple definitions in dictionary at byte 0x106a96 for key /Im194
Multiple definitions in dictionary at byte 0x106ab4 for key /Im194
Multiple definitions in dictionary at byte 0x106ad2 for key /Im194
Multiple definitions in dictionary at byte 0x106af0 for key /Im194
Multiple definiti

Лаборатория Касперского_543.pdf ПАО «Уралкалий»_1258.pdf ПАО «ТОАЗ»_1205.pdf ПАО «Россети»_958.pdf ОАО «Рязанская ГРЭС»_1017.pdf ОАО «Рязанская ГРЭС»_1016.pdf ПАО «ВЫМПЕЛКОМ» _ БРЕНД БИЛАЙН_285.pdf ПАО «ОГК-2»_785.pdf ОАО «Рязанская ГРЭС»_1014.pdf ОАО «Рязанская ГРЭС»_1015.pdf ПАО «ОГК-2»_784.pdf ОАО «РЖД»_902.pdf ОАО «Фортум» (ОАО «ТГК-10»)_1328.pdf ОАО «Фортум» (ОАО «ТГК-10»)_1329.pdf Владивостокский государственный университет экономики и сервиса_307.pdf ООО «НОВОГОР- Прикамье»_749.pdf ПАО «Татнефть»_1233.pdf ПАО «Газпром»_374.pdf Объединение «Желдортранс»_820.pdf ОАО «Енисейская территориальная генерирующая компания» (ТГК-13)_433.pdf ОАО «Енисейская территориальная генерирующая компания» (ТГК-13)_436.pdf ПАО «Газпром»_372.pdf ПАО «Газпром»_373.pdf АО «ОКБМ им.И.И.Африкантова»_554.pdf ОАО «ОГК-6»_791.pdf ОАО «Архангельский целлюлозно- бумажный комбинат» _203.pdf АО ИК «АСЭ» (АО «НИАЭП»)_453.pdf ОАО «Архангельский целлюлозно- бумажный комбинат» _202.pdf «Россети Северо-Запад»_961.pdf

In [33]:
for report_page_name in tqdm(os.listdir(PAGES_DIRECTORY)):
    os.rename(f'{PAGES_DIRECTORY}/{report_page_name}', f'{PAGES_DIRECTORY}/{report_page_name.replace(".pdf_", "_")}')

100%|█████████████████████████████████| 129886/129886 [00:29<00:00, 4476.79it/s]


In [34]:
os.listdir(PAGES_DIRECTORY)[:5]

['316_152.pdf', '4_81.pdf', '802_176.pdf', '665_49.pdf', '925_48.pdf']

In [3]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [4]:
cp_names =  {'ак',
             'акб',
             'акига',
             'алмазэргиэнбанк',
             'алрос',
             'альфабанк',
             'алюминиевый',
             'амвэя',
             'ангарский',
             'анк',
             'ао',
             'архангельский',
             'ас',
             'асэ',
             'атомпроект',
             'атомредметзолото',
             'атомэнергомаш',
             'атомэнергопроект',
             'атомэнергопром',
             'афк',
             'ашан',
             'аэрофлот',
             'балтика',
             'барс',
             'бат',
             'башнефть',
             'билайн',
             'бифид',
             'бф',
             'виктория',
             'владивостокский',
             'внипиэт',
             'волга',
             'волгателек',
             'волжский',
             'восток',
             'втб',
             'выксунский',
             'вымпелком',
             'вэб',
             'газпром',
             'газпромбанк',
             'гарантинвест',
             'генерировать',
             'геннадий',
             'гидропресс',
             'гк',
             'гмк',
             'гнц',
             'головной',
             'гольцблат',
             'грэс',
             'гтлк',
             'губкин',
             'дания',
             'дивизион',
             'евразийский',
             'еврохим',
             'елена',
             'енисейский',
             'еэс',
             'желдортранс',
             'зао',
             'зарубежнефть',
             'ик',
             'илим',
             'имииафрикантовый',
             'инжиниринговый',
             'инккапитал',
             'интер',
             'интернэшнл',
             'информация',
             'иркутск',
             'казаньоргсинтез',
             'казмунайгаз',
             'калининградский',
             'камаз',
             'касперский',
             'кб',
             'кемикалс',
             'ковровский',
             'комстаротс',
             'красцветмет',
             'крафтфудс',
             'кубань',
             'кузбассэнерго',
             'ленэнерго',
             'леруа',
             'лпк',
             'лср',
             'лукойл',
             'мвидео',
             'мдмбанк',
             'мегафон',
             'мерлен',
             'металлоинвест',
             'металлургический',
             'метафракс',
             'ммк',
             'монди',
             'мон’дэлиса',
             'моррис',
             'мосводоканал',
             'московский',
             'мрск',
             'мрф',
             'мсп',
             'мтс',
             'мхк',
             'неманский',
             'нестля',
             'ниаэп',
             'нижнекамскнефтехим',
             'нииар',
             'нипигаз',
             'ниу',
             'нк',
             'нлмк',
             'новатэк',
             'новогор',
             'новосибирский',
             'нордиск',
             'норильский',
             'оао',
             'объединение',
             'огк',
             'окб',
             'окбм',
             'около',
             'омк',
             'они',
             'ооо',
             'пао',
             'пейп',
             'пепеляев',
             'петропавловск',
             'пивоваренный',
             'пивоваров',
             'пигмент',
             'пик',
             'по',
             'полиметалл',
             'приволжье',
             'прикамье',
             'рао',
             'распадский',
             'ргу',
             'реновый',
             'ржд',
             'рксхолдинг',
             'рольф',
             'росатом',
             'росводоканал',
             'роснефть',
             'россеть',
             'российский',
             'россия',
             'ростелеком',
             'ростёха',
             'росэнергоатом',
             'рудна',
             'русало',
             'русгидро',
             'руссинвест',
             'руссия',
             'русый',
             'русь',
             'рф',
             'рязанский',
             'сабмиллер',
             'санофироссия',
             'сахалин',
             'сбербанк',
             'свез',
             'севергазпром',
             'севернефтегазпром',
             'северсталь',
             'сзлк',
             'сибирь',
             'сибурхолдинг',
             'система',
             'ситибанк',
             'ситроникс',
             'совкомбанк',
             'спбаэп',
             'суал',
             'сургутнефтегаз',
             'суэк',
             'схк',
             'сыктывкарский',
             'такед',
             'таманьнефтегаз',
             'татнефть',
             'твэл',
             'тгк',
             'техснабэкспорт',
             'тимченко',
             'тмк',
             'тнк',
             'тнквр',
             'тоаз',
             'трансаэро',
             'трансгаз',
             'транснефть',
             'тюмень',
             'урал',
             'уралкалий',
             'уралсиб',
             'уральский',
             'урсабанк',
             'ухта',
             'фгк',
             'ферреро',
             'фиабанк',
             'филиал',
             'филип',
             'фк',
             'фортум',
             'фосагро',
             'фпк',
             'фск',
             'химконцентрат',
             'центринвест',
             'черкизовый',
             'чусовской',
             'шелл',
             'шереметьево',
             'щуровский',
             'эксонмобила',
             'энела',
             'энергохолдинг',
             'энерджи',
             'эсэфай',
             'южнороссийский',
             'юкос',
             'юникредитбанк',
             'юнипро',
             'яндекс',
             'ятэк'}

In [5]:
PAGES_DIRECTORY_KEK = os.listdir(PAGES_DIRECTORY)[56300:].copy()

In [6]:
def parse_pages_to_texts():
    errors = []
    for report_page_name in tqdm(PAGES_DIRECTORY_KEK):
        if not report_page_name.endswith('.pdf'):
            continue
        try:
            report_page = textract.process(f'{PAGES_DIRECTORY}/{report_page_name}')
        except Exception:
            errors.append(report_page_name)
            continue
        report_page = report_page.decode().replace('\n', ' ')
        report_page = RE_DERIVED.sub('', report_page)
        report_page_lst = word_tokenize(report_page)
        # russian_report_page_lst = [w for w in filter(RE_RUSSIAN_TEXT.match, report_page_lst)]
        tokens = []
        for word_ in filter(RE_RUSSIAN_TEXT.match, report_page_lst):
            if word_ and word_ not in stopwords_ru and word_ not in cp_names:
                word_ = word_.strip()
                word_ = morph.normal_forms(word_)[0]
                tokens.append(word_)
        russian_report_page_txt = ' '.join(tokens)
        if not russian_report_page_txt:
            continue
        with open(f'{PAGES_TXT_DIRECTORY}/{report_page_name.replace(".pdf", ".txt")}', 'w') as f:
            f.write(russian_report_page_txt)
    print(*errors, sep='\n')

In [7]:
parse_pages_to_texts()

100%|██████████████████████████████████| 73586/73586 [89:12:51<00:00,  4.36s/it]

133_24.pdf
133_30.pdf
383_39.pdf
133_18.pdf
383_11.pdf
1220_98.pdf
119_5.pdf
383_13.pdf
133_26.pdf
133_32.pdf
90_48.pdf
1220_59.pdf
133_129.pdf
863_4.pdf
530_41.pdf
1097_59.pdf
523_29.pdf
90_6.pdf
530_55.pdf
133_115.pdf
1220_65.pdf
1097_65.pdf
1097_71.pdf
13_0.pdf
523_15.pdf
1220_71.pdf
133_101.pdf
530_69.pdf
873_42.pdf
873_56.pdf
344_28.pdf
874_63.pdf
344_14.pdf
861_6.pdf
92_4.pdf
926_1.pdf
640_46.pdf
699_41.pdf
640_52.pdf
1221_139.pdf
876_72.pdf
640_8.pdf
1221_105.pdf
296_26.pdf
1220_1.pdf
296_32.pdf
69_16.pdf
1221_111.pdf
876_66.pdf
859_85.pdf
69_2.pdf
13_37.pdf
522_55.pdf
1221_31.pdf
861_101.pdf
1221_25.pdf
861_115.pdf
876_99.pdf
13_23.pdf
522_41.pdf
640_160.pdf
1098_20.pdf
859_52.pdf
640_148.pdf
1221_19.pdf
861_129.pdf
1098_34.pdf
1220_128.pdf
861_56.pdf
875_23.pdf
875_37.pdf
861_42.pdf
1077_26.pdf
1220_114.pdf
1219_35.pdf
872_16.pdf
1219_21.pdf
1220_100.pdf
1077_32.pdf
1222_3.pdf
699_9.pdf
521_38.pdf
861_95.pdf
876_8.pdf
1222_48.pdf
861_81.pdf
1219_5.pdf
1222_60.pdf
1222_74.pdf
5

In [11]:
a = len(os.listdir(PAGES_TXT_DIRECTORY))
b = len(os.listdir(PAGES_DIRECTORY))

In [12]:
1 - a/b

0.08834670403276723

In [13]:
a, b

(118411, 129886)

In [14]:
a/b

0.9116532959672328

In [17]:
rep_words_freq = defaultdict(int) # в скольких отчетах употребляется слово

In [18]:
for report in tqdm(os.listdir(PAGES_TXT_DIRECTORY)):
    if not report.endswith('.txt'):
        continue

    report_filename = f'{PAGES_TXT_DIRECTORY}/{report}'
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)

    report_words = set()
    for word in report_text_lst:
        if not word in report_words:
            report_words.add(word)
            rep_words_freq[word] += 1

100%|██████████████████████████████████| 118411/118411 [02:30<00:00, 785.50it/s]


In [19]:
amount_of_pages = len(os.listdir(PAGES_TXT_DIRECTORY))
rep_words_freq_percent = {k: v / amount_of_pages for (k, v) in rep_words_freq.items()}

In [20]:
rep_words_freq_percent

{'период': 0.2523414209828479,
 'летний': 0.01421320654331101,
 'межень': 0.0001182322588273049,
 'в': 0.6759929398451158,
 'обычный': 0.011029380716318586,
 'существенный': 0.13969141380446073,
 'влияние': 0.10435685873778619,
 'наблюдаться': 0.01087736781211205,
 'предел': 0.03423668409184957,
 'участок': 0.04562920674599488,
 'недропользование': 0.0022886387244428303,
 'добыча': 0.0625617552423339,
 'питьевой': 0.007085490368293486,
 'вода': 0.08693449088344833,
 'подземный': 0.022742819501566577,
 'скважина': 0.01932252915691954,
 'образоваться': 0.017135232368614403,
 'депрессионный': 1.6890322689614984e-05,
 'воронка': 0.00043070322858518214,
 'сведение': 0.07491702628978726,
 'общий': 0.3127749955662903,
 'количество': 0.19461874319108866,
 'забирать': 0.006958812948121374,
 'источник': 0.09598770384508196,
 'местонахождение': 0.006376096815329657,
 'таблица': 0.061218974588509514,
 'объём': 0.23328913698896217,
 'водопотребление': 0.012574845242418356,
 'тыс': 0.157569820371418

In [22]:
min(list(rep_words_freq_percent.values())), max(list(rep_words_freq_percent.values()))

(8.445161344807492e-06, 0.7362153853949379)

In [64]:
import numpy as np
import pandas as pd

In [86]:
v = np.array(list(rep_words_freq_percent.values()))
f'{np.percentile(v, 85):.6f}'

'0.000110'

In [78]:
between_ = set()
for key, value in tqdm(rep_words_freq_percent.items()):
    if value >= 0.000878 and value <= 0.3:
        between_.add(key)

100%|██████████████████████████████| 183749/183749 [00:00<00:00, 2749619.74it/s]


In [79]:
len(between_)

9177

In [65]:
pages_txt = {'report_num': [],
             'page_num': [],
             'text': []}

In [66]:
os.listdir(PAGES_TXT_DIRECTORY)[:5]

['341_187.txt', '1173_9.txt', '49_75.txt', '1253_99.txt', '942_64.txt']

In [82]:
for report in tqdm(os.listdir(PAGES_TXT_DIRECTORY)):
    if not report.endswith('.txt'):
        continue
    report_filename = f'{PAGES_TXT_DIRECTORY}/{report}'
    with open(report_filename) as f:
        report_text = f.read()
    report_text_lst = word_tokenize(report_text)
    report_words = [report_word for report_word in report_text_lst if report_word in between_]
    if len(report_words) == 0:
        continue
    report_num, page_num = get_report_and_page_ids(report)
    pages_txt['report_num'].append(report_num)
    pages_txt['page_num'].append(page_num)
    pages_txt['text'].append(' '.join(report_words))

100%|██████████████████████████████████| 118411/118411 [02:27<00:00, 802.58it/s]


In [83]:
pages_df = pd.DataFrame.from_dict(pages_txt)
pages_df.head()

,report_num,page_num,text
0,341,187,период летний обычный период существенный влия...
1,1173,9,основной направление природоохранный экология ...
2,49,75,мнение заинтересованный сторона внутренний вне...
3,1253,99,действие сфера культура спор тот г. средство в...
4,942,64,л


In [84]:
pages_df.shape

(118157, 3)

In [85]:
pages_df.to_csv('report_pages_ds.csv')